In [ ]:
#!pip3 install sentence-transformers

In [ ]:
#!pip3 install pinecone

In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/medium_data.csv

--2025-11-17 22:16:14--  https://s3-geospatial.s3.us-west-2.amazonaws.com/medium_data.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 3.5.78.129, 3.5.86.77, 52.92.205.210, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|3.5.78.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 599580 (586K) [text/csv]
Saving to: ‘medium_data.csv’

medium_data.csv     100%[===================>] 585.53K  3.44MB/s    in 0.2s    

2025-11-17 22:16:14 (3.44 MB/s) - ‘medium_data.csv’ saved [599580/599580]



In [ ]:
!wc -l medium_data.csv

2499 medium_data.csv


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("medium_data.csv") # excercise whole data set

In [ ]:
df.head()


,id,url,title,subtitle,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",NaN,188.0,0,24,Towards Data Science,27-01-2023
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,NaN,10.0,0,13,Towards Data Science,27-01-2023


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2498 entries, 0 to 2497
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            2498 non-null   int64  
 1   url           2498 non-null   object 
 2   title         2498 non-null   object 
 3   subtitle      2073 non-null   object 
 4   claps         2423 non-null   float64
 5   responses     2498 non-null   int64  
 6   reading_time  2498 non-null   int64  
 7   publication   2498 non-null   object 
 8   date          2498 non-null   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 175.8+ KB


In [ ]:
df['title'] = df['title'].astype(str).fillna('')
df['subtitle'] = df['subtitle'].astype(str).fillna('')

In [ ]:
df['metadata'] = df.apply(lambda row: {'title': row['title'] + " " + row['subtitle']}, axis=1)

In [ ]:
from google.colab import userdata

# Use Colab Secrets
API_KEY = userdata.get('pinecone_key')

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key=API_KEY)

In [ ]:
from pinecone import ServerlessSpec

# Spec for free tier plan
spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)


In [ ]:
index_name = 'semantic-search-fast'

In [ ]:
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

In [ ]:
existing_indexes

[]

In [ ]:
import time

# If the following code runs more than once, you might get "The index already existed" error
# In that case, delete the index first: pc.delete_index(index_name)
try:
    pc.delete_index(index_name)
except Exception as e:
    print(e)

pc.create_index(
    index_name,
    dimension=384,  # dimensionality of minilm
    metric='dotproduct',
    spec=spec
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu') # cuda or cpu

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# .apply(lambda x: ...): This applies a function to each row in the 'metadata' column.
# model.encode(x["title"]): This uses a sentence embedding model to encode the text in the "title" field of x into a vector (embedding).
# .tolist(): Converts the embedding (a NumPy array) to a regular Python list.
# df['values'] = df['metadata'].map(
#    lambda x: (model.encode(x["title"])).tolist())
df['values'] = df['metadata'].apply(
    lambda x: model.encode(x['title']).tolist()) # python list, 6k rows 1 min

In [ ]:
df.head()

,id,url,title,subtitle,claps,responses,reading_time,publication,date,metadata,values
0,1,https://towardsdatascience.com/not-all-rainbow...,Not All Rainbows and Sunshine: The Darker Side...,Part 1: The Risks and Ethical Issues…,453.0,11,9,Towards Data Science,27-01-2023,{'title': 'Not All Rainbows and Sunshine: The ...,"[0.004084164742380381, 0.03849547728896141, 0...."
1,2,https://towardsdatascience.com/ethics-in-ai-po...,Ethics in AI: Potential Root Causes for Biased...,An alternative approach to understanding bias ...,311.0,3,12,Towards Data Science,27-01-2023,{'title': 'Ethics in AI: Potential Root Causes...,"[-0.0701502114534378, 0.02475568652153015, -0...."
2,3,https://towardsdatascience.com/python-tuple-th...,"Python Tuple, The Whole Truth and Only the Tru...",nan,188.0,0,24,Towards Data Science,27-01-2023,"{'title': 'Python Tuple, The Whole Truth and O...","[-0.0594332255423069, -0.027460744604468346, 0..."
3,4,https://towardsdatascience.com/dates-and-subqu...,Dates and Subqueries in SQL,Working with dates in SQL,15.0,1,4,Towards Data Science,27-01-2023,{'title': 'Dates and Subqueries in SQL Working...,"[0.002338154474273324, -0.003287985920906067, ..."
4,5,https://towardsdatascience.com/temporal-differ...,Temporal Differences with Python: First Sample...,nan,10.0,0,13,Towards Data Science,27-01-2023,{'title': 'Temporal Differences with Python: F...,"[-0.08383705466985703, -0.02956460975110531, 0..."


In [ ]:
# 'values', 'metadata', 'sparse_values', 'id'
df_upsert = df[['id', 'values', 'metadata']]

In [ ]:
# df_upsert['id'] = df_upsert['id'].map(lambda x: str(x))
df_upsert['id'] = df_upsert['id'].astype(str)

/tmp/ipython-input-3159772791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upsert['id'] = df_upsert['id'].astype(str)


In [ ]:
df_upsert.head()

,id,values,metadata
0,1,"[0.004084164742380381, 0.03849547728896141, 0....",{'title': 'Not All Rainbows and Sunshine: The ...
1,2,"[-0.0701502114534378, 0.02475568652153015, -0....",{'title': 'Ethics in AI: Potential Root Causes...
2,3,"[-0.0594332255423069, -0.027460744604468346, 0...","{'title': 'Python Tuple, The Whole Truth and O..."
3,4,"[0.002338154474273324, -0.003287985920906067, ...",{'title': 'Dates and Subqueries in SQL Working...
4,5,"[-0.08383705466985703, -0.02956460975110531, 0...",{'title': 'Temporal Differences with Python: F...


In [ ]:
index = pc.Index(index_name)

In [ ]:
xc = index.query(vector=(model.encode("what is ethics in AI")).tolist(), # python list
           top_k=10,
           include_metadata=True,
           include_values=True)   #

In [ ]:
for result in xc['matches']:
    print(result['id'], result['score'], result['metadata'])